In [1]:
# Importando as bibliotecas
import bs4 
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import Request, urlopen, urlretrieve
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import pandas

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pandas.__version__)

BeautifulSoup -> 4.10.0
urllib -> 3.9
pandas -> 1.3.4


In [2]:
zap_imoveis = 'https://www.zapimoveis.com.br/venda/?pagina=1&transacao=Venda&tipo=Im%C3%B3vel%20usado'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

requisicao = Request(zap_imoveis,headers = headers)

try:
    response = urlopen(requisicao)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    print(soup)
except HTTPError as e:
    print(e.status(), e.reason())
except URLError as e:
    print(e.reason())    

<!DOCTYPE html>

<html data-vue-meta-server-rendered="" lang="pt-br">
<head>
<meta charset="utf-8" data-vue-meta="ssr"/><meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0" data-vue-meta="ssr" name="viewport"/><meta content="#6e0ad6" data-vue-meta="ssr" name="theme-color"/><meta content="No ZAP Imóveis você encontra os melhores anúncios de Imóveis à venda em todo o Brasil. Acesse já e feche ótimos negócios!" data-vue-meta="ssr" name="description"/><meta content="app-id=514620313, app-clip-bundle-id=br.com.zap.ZAPImoveis.AppClip, app-clip-display=card" data-vue-meta="ssr" name="apple-itunes-app"/><meta content="Imóveis à venda - Ofertas de Imóveis à venda - ZAP Imóveis" data-vue-meta="ssr" name="og:title"/><meta content="https://assets.zap.com.br/assets/v5.165.0/zap-imoveis-ok.png?96b961897e3730b77e3967ce9c64032e" data-vue-meta="ssr" name="og:image"/><meta content="No ZAP Imóveis você encontra os melhores anúncios de Imóveis à venda em todo o Brasil. Acesse já e fech

In [128]:
#Declarando variável cards
cards = []


##Obtendo HTML e o total de paginas
# response = urlopen('https://www.zapimoveis.com.br/venda/?pagina=3&transacao=Venda&tipo=Im%C3%B3vel%20usado')
# html = response.read().decode('utf-8')
# soup = BeautifulSoup(html,'html.parser')
# pages = int(20)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/?pagina=' + str(i)'&transacao=Venda&tipo=Im%C3%B3vel%20usado'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-container')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        card['value'] = anuncio.find('p', {'class': 'simple-card__price'}).getText()

        # Informações
        infos = anuncio.find('div',{'class':'body-card'}).findAll('p')
        for info in infos:
            card[info.get('class')[0].split('-')[-1]] = info.get_text()

        # Acessorios
        items = anuncio.find('div',{'class':'body-card'}).ul.findAll('li')
        items.pop()
        acessorios = []
        for item in items:
            acessorios.append(item.getText().replace('► ',''))
        card['items'] = acessorios 

        #Adicionando resultado a lista cards
        cards.append(card)

        # Imagens
        image = anuncio.find('div',{'class':'image-card'}).img
        nome_imagem = image.get('src').split('/')[-1]
        urlretrieve(image.get('src'),'./output/img/' + nome_imagem)


dataset = pd.DataFrame(cards)
dataset.to_csv('./output/data/dataset.csv',sep=';', index=False,encoding='utf-8-sig')
dataset

SyntaxError: invalid syntax (987284305.py, line 16)

In [129]:
#Declarando variável cards
cards = []


##Obtendo HTML e o total de paginas
# response = urlopen('https://www.zapimoveis.com.br/venda/?pagina=3&transacao=Venda&tipo=Im%C3%B3vel%20usado')
# html = response.read().decode('utf-8')
# soup = BeautifulSoup(html,'html.parser')
pages = int(1)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/?pagina=' + str(i + 1)+ '&transacao=Venda&tipo=Im%C3%B3vel%20usado'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        card['value'] = anuncio.find('p', {'class': 'simple-card__price'}).getText().replace(' ','').replace('R$','').replace('\n','')

        #descricao
        card['descricao'] = anuncio.find('span',{'class':'simple-card__text text-regular'}).get_text()
        
        #endereco
        card['endereco'] = anuncio.find('div',{'class':'simple-card__actions'}).h2.get_text()
        
        #banheiro
        #card['banheiro'] = anuncio.find('li',{'class':'feature__item text-small js-bathrooms'}).get_text()
        
        cards.append(card)
        
print(cards)        

[{'value': '420.000', 'descricao': 'O SOBRADO    Sobrado casa com dois quartos, dormitorios, suites e duas vagas de garagem, e churrasqueira, com area construída de oitenta e cinco metros quadrados, com alarme e cerc...', 'endereco': '\n          Rua 234, Meia Praia\n        '}, {'value': '995.000', 'descricao': 'Apartamento garden de alto padrão, com 2 dormitórios ,1 suíte c/ventilação direta, 2 vagas cobertas e escrituradas , no bairro Petrópolis. O imóvel possui living amplo c/2 ambien...', 'endereco': '\n          Petrópolis, Porto Alegre\n        '}, {'value': '210.000', 'descricao': 'Maravilhoso apartamento amplo com cômodos arejados no bairro da Vila Tupi, com 48 metros de área útil distribuídos em 01 dormitório, sacada e suas demais dependências. Residencial...', 'endereco': '\n          Vila Tupi, Praia Grande\n        '}, {'value': '740.000', 'descricao': 'Luxo e conforto localizado perto do mar e do metrô. Proprietário vende excelente apartamento, todo reformado e decorado, 

In [ ]:
for preco in soup.find('div',{'class':'listings__container'}).findAll('p',{'class':'simple-card__price'}): 
    precos = []
    precos = preco.get_text().replace(' ','').replace('R$','')
    print(precos)


# soup.find('p',{'class':'simple-card__price'}).get_text()#.replace('\n','').replace(' ','')

In [ ]:
soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-container')

In [ ]:
soup.find('p',{'class':'simple-card__price'}).get_text()#.replace('\n','').replace(' ','')

In [48]:
for anuncio in soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card'):
    print(anuncio.find('span',{'class':'simple-card__text text-regular'}).get_text())

(Cód: 6664MVAYY) Vende-se Casa (Diversos) no bairro Jardim Zara (Zona Leste), área construída de 187 m2, em terreno de 200 m2, 3 dormitório(s) sendo 1 suite(s), 2 banheiro(s), 2 ga...
2501
Casa rústica em ótimo terreno localizada em Cacupé com linda vista. Localizaç?o privilegiada, vista livre para o mar e rodeada pela natureza. 3 quartos, 2 banheiros, garagem  Marqu...
Apartamento na Vila Romana, andar alto, duas vagas de garagem + depósito privativo, vista desbloqueada, prédio novo, excelente localização e próximo a estação da Lapa. 68 m2 ,dois...
Apartamento bem localizado, próximo ao Shopping Dom Pedro, único com ar condicionado em todos os ambientes, porcelanato, varanda gourmet, armários planejados, despensa , lazer comp...
Unidade 165, Bloco 1, frente para a avenida Sezefredo Fagundes, vista espetacular, armários embutidos com 2 anos de uso, excelente estado de conservação (apenas 6 anos), área de la...
Apartamento no 25º andar, vista praia - porto no canal 6. Com todos os itens

In [134]:
for anuncio in soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card'):
        items = anuncio.find('div',{'class':'simple-card__actions'}).ul.findAll('li')
        #items.pop()
        acessorios = []
        for item in items:
            acessorios.append(item.get_text().replace('\n','').replace(' ',''))
            print(acessorios)
        
    
    #print(anuncio.find('span', {'itemprop':'numberOfBathroomsTotal'}))

['85m²']
['85m²', '2']
['85m²', '2', '2']
['85m²', '2', '2', '3']
['106m²']
['106m²', '2']
['106m²', '2', '2']
['106m²', '2', '2', '2']
['48m²']
['48m²', '1']
['48m²', '1', '1']
['48m²', '1', '1', '1']
['75m²']
['75m²', '2']
['75m²', '2', '2']
['106m²']
['106m²', '3']
['106m²', '3', '2']
['106m²', '3', '2', '3']
['654m²']
['654m²', '5']
['654m²', '5', '2']
['654m²', '5', '2', '7']
['276m²']
['276m²', '4']
['276m²', '4', '4']
['276m²', '4', '4', '6']
['152m²']
['152m²', '3']
['152m²', '3', '2']
['152m²', '3', '2', '4']
['150m²']
['150m²', '3']
['150m²', '3', '3']
['150m²', '3', '3', '5']
['275m²']
['275m²', '5']
['275m²', '5', '3']
['275m²', '5', '3', '5']
['83m²']
['83m²', '3']
['83m²', '3', '2']
['83m²', '3', '2', '2']
['78m²']
['78m²', '2']
['78m²', '2', '2']
['78m²', '2', '2', '2']
['85m²']
['85m²', '2']
['85m²', '2', '2']
['85m²', '2', '2', '2']
['72m²']
['72m²', '2']
['72m²', '2', '1']
['72m²', '2', '1', '3']
['93m²']
['93m²', '3']
['93m²', '3', '2']
['93m²', '3', '2', '3']
['172m